# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'



In [26]:
# Using techniques from mysql-examples notebook:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [3]:
host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'

In [4]:
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [5]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [6]:
# URL provided cues download of a CSV file
# Read in the file to a pandas dataframe
data = pd.read_csv('Electronic_Health_Record_EHR_Incentive_Program_Payments_to_Eligible_Hospitals.csv')

# Check the data 
data.head

<bound method NDFrame.head of               X          Y  OBJECTID  \
0   -124.142009  40.783559         1   
1   -122.086674  37.632915         2   
2   -122.295861  38.325402         3   
3   -120.766467  38.350920         4   
4   -122.435820  37.769049         5   
..          ...        ...       ...   
318 -122.173214  37.436328       319   
319 -118.186632  33.808293       320   
320 -121.455450  38.553012       321   
321 -118.286401  34.067178       322   
322 -117.152024  32.798322       323   

                                         Provider_Name         NPI    CCN  \
0             ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC  1609858950  50006   
1                             HAYWARD SISTERS HOSPITAL  1942298153  50002   
2             ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC  1235218785  50009   
3                              SUTTER VALLEY HOSPITALS  1447494323  50014   
4                                 Sutter Bay Hospitals  1659439834  50008   
..                         

In [7]:
# Import the data to SQL using the MYTABLE variable created above
data.to_sql(MYTABLE, conn, index = True)

ValueError: Table 'ecdmdphd_data' already exists.

In [8]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [9]:
# Use SQL to join the "population" table with "MYTABLE" by zip code
mytable_pop = pd.read_sql_query("""
  SELECT m. *, p.*
  FROM 
    ecdmdphd_data m INNER JOIN
    population p ON m.business_zip_code = p.zip
  """, conn)

In [10]:
mytable_pop.columns

Index(['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year', 'index', 'Zip',
       'Population'],
      dtype='object')

In [11]:
# Determine which providers are located in the zipcode with the largest population
# Find the maximum population
MaxPop = mytable_pop['Population'] == mytable_pop['Population'].max()

In [12]:
# Filter the dataset to those rows with the largest population
MaxPopZip = mytable_pop[MaxPop]['NPI']

In [13]:
MaxPopZip

0    1194016923
Name: NPI, dtype: int64

In [14]:
# Return a list of NPI for that zipcode
answer = []
for r in MaxPopZip:
    npi = str(r)
    answer.append(npi)

In [15]:
answer

['1194016923']

In [16]:
assert(type(answer) == list)
assert(answer == ['1194016923'])


## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per capita payment for these type of hospital.
    * average the total payments and divide by population

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [17]:
# Create a Grouped data frame by zip code and hospital type, summing the total payments and joining with population data
payments = pd.read_sql_query('''
SELECT m.total_payments, m.Business_ZIP_code, m.Medicaid_EP_Hospital_Type, p.Population, p.Zip
FROM 
    ecdmdphd_data m INNER JOIN
    population p ON m.business_zip_code = p.zip
GROUP BY Zip, Medicaid_EP_Hospital_Type
''', conn)

In [18]:
payments

,total_payments,Business_ZIP_code,Medicaid_EP_Hospital_Type,Population,Zip
0,1093021.73,90004,Acute Care Hospitals,61105,90004
1,3614739.93,90012,Acute Care Hospitals,37268,90012
2,10045484.04,90015,Acute Care Hospitals,23900,90015
3,3899845.21,90017,Acute Care Hospitals,27832,90017
4,427640.22,90020,Children's Hospitals,39366,90020
...,...,...,...,...,...
256,184615.39,96104,Acute Care Hospitals,920,96104
257,531907.08,96122,Acute Care Hospitals,3594,96122
258,1371962.88,96130,Acute Care Hospitals,19859,96130
259,786769.13,96150,Acute Care Hospitals,30369,96150


In [19]:
# Using Pandas, modify the data frame 
# Calculate a total payments per person column
payments['PerPerson'] = payments['total_payments']/payments['Population']

In [25]:
payments

,total_payments,Business_ZIP_code,Medicaid_EP_Hospital_Type,Population,Zip,PerPerson
0,1093021.73,90004,Acute Care Hospitals,61105,90004,17.887599
1,3614739.93,90012,Acute Care Hospitals,37268,90012,96.993129
2,10045484.04,90015,Acute Care Hospitals,23900,90015,420.313140
3,3899845.21,90017,Acute Care Hospitals,27832,90017,140.120912
4,427640.22,90020,Children's Hospitals,39366,90020,10.863187
...,...,...,...,...,...,...
256,184615.39,96104,Acute Care Hospitals,920,96104,200.668902
257,531907.08,96122,Acute Care Hospitals,3594,96122,147.998631
258,1371962.88,96130,Acute Care Hospitals,19859,96130,69.085195
259,786769.13,96150,Acute Care Hospitals,30369,96150,25.906982


In [27]:
# Average across each Medicaid hospital type by creating a Pivot table
answer = pd.pivot_table(data = payments, values = 'PerPerson', columns = 'Medicaid_EP_Hospital_Type', aggfunc = np.mean)

In [28]:
answer

Medicaid_EP_Hospital_Type,Acute Care Hospitals,Children's Hospitals
PerPerson,87.0352,324.265318


In [23]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
answer

## Now using Pandas instead of SQL

In [29]:
import numpy as np

In [30]:
# Using the dataframe created in step 3, mytable_pop, compute total payments for zip code and hospital type
grouped_table = mytable_pop.groupby(by =['Zip', 'Medicaid_EP_Hospital_Type'])

In [37]:
grouped_table = grouped_table['total_payments'].agg(np.mean)

In [38]:
grouped_table

Zip    Medicaid_EP_Hospital_Type
90004  Acute Care Hospitals          1093021.73
90012  Acute Care Hospitals          3614739.93
90015  Acute Care Hospitals         10045484.04
90017  Acute Care Hospitals          3899845.21
90020  Children's Hospitals           427640.22
                                       ...     
96104  Acute Care Hospitals           184615.39
96122  Acute Care Hospitals           531907.08
96130  Acute Care Hospitals          1371962.88
96150  Acute Care Hospitals           786769.13
96161  Acute Care Hospitals           621315.39
Name: total_payments, Length: 261, dtype: float64

In [32]:
# Calculate a total payments per person column
payments['PerPerson'] = payments['total_payments']/payments['Population']

In [ ]:
payments['PerPerson']

In [ ]:
# Average across each Medicaid hospital type by creating a Pivot table
answer = pd.pivot_table(data = payments, values = 'PerPerson', columns = 'Medicaid_EP_Hospital_Type', aggfunc = 'mean')

In [ ]:
answer

### A quick note & apology
I know that the answers above do not match the assertions. I made an attempt here and ultimately decided to concentrate my time on the final project, so I can finish with classes this week in time for a trip home to see my family (including accompanying my mom and grandmother to a procedure for my grandmother and my younger brother's senior recital, the last requirement of his college degree). I look forward to reviewing the solution to see what I could have done differently. 



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [39]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


Already up to date.
[main 157b813] Submitting the week 15 programming assignment
 2 files changed, 2299 insertions(+), 17 deletions(-)
 create mode 100644 week15/week15_assignment.ipynb
Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 19.03 KiB | 4.76 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:ecdmdphd/hds5210-2022.git
   d2603e9..157b813  main -> main
